### This notebook aims to outline the steps needed to transform an e-commerce fraud dataset using a Data-Centric approach.

### Based on the following source:

#### https://dcai.csail.mit.edu/2024/data-centric-model-centric/

### And paraphrasing:

<br>

### <i>".. model-centric AI is based on the goal of producing the best model for a given dataset, whereas data-centric AI is based on the goal of systematically & algorithmically producing the best dataset to feed a given ML model. To deploy the best supervised learning systems in practice, one should do both."</i>

<br>

### This means the dataset must be adapted so it can be easily used by an algorithm, in this case one related to classification due to the nature of the problem.

### In the following lines, the transformation is carried out step by step with its corresponding justification.

<br>

### <b> Note: the use of the dataset X_test_datapub.csv is automatically discarded because, especially during the analysis stage with the target, there is no way to see how the data correlates, so its use becomes futile. </b>

# Step 1. Loading Variables.

In [1]:
# Importing required libraries for data transformation.
import numpy as np
import pandas as pd

# For the correlation matrix.
import seaborn as sns
import matplotlib.pyplot as plt

# These libraries are for similarity between column names.
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amcm3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\amcm3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amcm3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amcm3\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\amcm3\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Step 2. Loading Training Dataframes. 

In [2]:
# We read the X_train dataframe.
part1_csv = "train/X_train_datapub_part1.csv"
part2_csv = "train/X_train_datapub_part2.csv"

df_part1 = pd.read_csv(part1_csv)
df_part2 = pd.read_csv(part2_csv)

print(f"Part 1 shape: {df_part1.shape}")
print(f"Part 2 shape: {df_part2.shape}")

train = pd.concat([df_part1, df_part2], axis=0, ignore_index=True)

C:\Users\amcm3\AppData\Local\Temp\ipykernel_15456\404204214.py:5: DtypeWarning: Columns (22,23,24,70,71,72,94,95,96,97,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_part1 = pd.read_csv(part1_csv)


Part 1 shape: (46395, 146)
Part 2 shape: (46395, 146)


C:\Users\amcm3\AppData\Local\Temp\ipykernel_15456\404204214.py:6: DtypeWarning: Columns (17,18,19,20,21,22,23,24,65,66,67,68,69,70,71,72,89,90,91,92,93,94,95,96,97,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_part2 = pd.read_csv(part2_csv)


In [3]:
# We read the target dataframe.
target = pd.read_csv("train/Y_train_datapub.csv")

# We store the name of the column corresponding to the predictive variable.
predictive_variable = "fraud_flag"

# 3.- Exploratory Data Analysis and Data Preprocessing

In [4]:
# We print the dataframe to start visualizing the data.
train

,ID,item1,item2,item3,item4,item5,item6,item7,item8,item9,...,Nbr_of_prod_purchas16,Nbr_of_prod_purchas17,Nbr_of_prod_purchas18,Nbr_of_prod_purchas19,Nbr_of_prod_purchas20,Nbr_of_prod_purchas21,Nbr_of_prod_purchas22,Nbr_of_prod_purchas23,Nbr_of_prod_purchas24,Nb_of_items
0,85517,COMPUTERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,51113,COMPUTER PERIPHERALS ACCESSORIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,83008,TELEVISIONS HOME CINEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,78712,COMPUTERS,COMPUTER PERIPHERALS ACCESSORIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
4,77846,TELEVISIONS HOME CINEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92785,21243,COMPUTER PERIPHERALS ACCESSORIES,FULFILMENT CHARGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
92786,45891,COMPUTERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
92787,42613,BEDROOM FURNITURE,BEDROOM FURNITURE,OUTDOOR FURNITURE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
92788,43567,COMPUTERS,COMPUTERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


### 1.- The first issue detected is related to the size of the product basket. Among other things, this limits the analysis to only 24 elements (the columns item, Nbr_of_prod_purchase, cash_price).

### The proposal is that each column be associated with products, because for a model it is more natural to focus on the product(s) that lead to fraud rather than on the product basket.

### The difference is that with the basket you would need to extract each product and then group them, while in the product-per-column solution you simply mark the quantity purchased, making the operations easier.

### This second approach has no limit on the number of products purchased, while the basket approach does.

<br>

### <b> Special note on data type: in the basket the products are still strings, while in the product-per-column solution a numeric transformation has already been done, which is much easier for any algorithm to use. </b>

<br>

### <b> Summary: the change is made to one column per product and the columns item1 to item24, as well as Nbr_of_prod_purchas1 to Nbr_of_prod_purchas24, are removed because they are no longer needed given the proposed column-per-product change. </b>
### <b> All purchase costs (variables cash_price1 to cash_price24) are also summarized so there is a single field (total_purchase) that contains the total value of the purchase, making it easier to analyze. </b>



In [5]:
# First, get all possible products stored in 
# the columns item1 a item24:

# Here we store products with their frequencies.
dict_items = {}

# We create a copy of the original dataframe in case we want to apply
# some imputation operation.
train_2 = train.copy()

# We count products for each itemx column.
for x in range (1,25):
    current_item = train["item{0}".format(x)].value_counts().index.tolist()
    for element in current_item:

        if element in dict_items:
           dict_items[element] += 1

        else:
           dict_items[element] = 1
print(" ")
print("How many unique products: ", len(dict_items))
print(" ")

# The second step is that, for each element, we create a proper unique key.
for current_key in dict_items.keys():
    formatted_key = "item_" + current_key.lower().replace(" ","_")
    train_2[formatted_key] = 0

print("Dataframe with aggregated columns: ")
print(train_2)

 
How many unique products:  173
 
Dataframe with aggregated columns: 
          ID                             item1  \
0      85517                         COMPUTERS   
1      51113  COMPUTER PERIPHERALS ACCESSORIES   
2      83008           TELEVISIONS HOME CINEMA   
3      78712                         COMPUTERS   
4      77846           TELEVISIONS HOME CINEMA   
...      ...                               ...   
92785  21243  COMPUTER PERIPHERALS ACCESSORIES   
92786  45891                         COMPUTERS   
92787  42613                 BEDROOM FURNITURE   
92788  43567                         COMPUTERS   
92789  68268           TELEVISIONS HOME CINEMA   

                                  item2              item3 item4 item5 item6  \
0                                   NaN                NaN   NaN   NaN   NaN   
1                                   NaN                NaN   NaN   NaN   NaN   
2                                   NaN                NaN   NaN   NaN   NaN   
3      C

C:\Users\amcm3\AppData\Local\Temp\ipykernel_15456\3483396113.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_2[formatted_key] = 0
C:\Users\amcm3\AppData\Local\Temp\ipykernel_15456\3483396113.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_2[formatted_key] = 0
C:\Users\amcm3\AppData\Local\Temp\ipykernel_15456\3483396113.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once 

In [6]:
def fill_items(row):
    """
    Convert per-row item slots (item1..item24) into aggregated item counters and total purchase value.

    This function scans the columns `item1` to `item24`. For each non-null string item value, it:
    1) Builds a counter column name like `item_<normalized_item_name>` (lowercase, spaces -> underscores).
    2) Increments that counter by 1 in the row.
    3) Adds the corresponding `cash_priceX` value to `total_purchase`.

    Args:
        row (pandas.Series): A row containing:
            - item1..item24 (object): Item identifiers. Only string values are processed.
            - cash_price1..cash_price24 (float or int): Price values corresponding to each item slot.
            - total_purchase (float or int): Accumulator that will be increased by matching cash_priceX.
            - item_<name> (int): One counter column per possible item name, pre-initialized (typically to 0).

    Returns:
        pandas.Series: The same row with updated `item_<name>` counters and `total_purchase`.

    Notes:
        - If a normalized counter column (e.g., `item_bread`) does not exist in `row`,
          the increment will raise an exception and the function will print a simple log message.
        - Non-string or null item values are ignored.
    """
    # We loop through fields item1 to item24.
    for x in range(1, 25):
        current_element = row["item{0}".format(x)]

        # If values are not null, then find the corresponding field
        # for the current row and update the counter by +1.
        try:
            if type(current_element) == str:
                formatted_current_element = "item_" + current_element.lower().replace(" ", "_")
                row[formatted_current_element] += 1

                row["total_purchase"] += row["cash_price{0}".format(x)]

        # If there is an error, print a simple log message.
        except:
            print(type(current_element), "-", current_element)

    return row

In [7]:
# We add the total_purchase function to summarize the cost variables
# per product.
train_2["total_purchase"] = 0

# We apply the fill_items function to train_2 and print to see results:
train_2 = train_2.apply(fill_items, axis = 1)
train_2

C:\Users\amcm3\AppData\Local\Temp\ipykernel_15456\4117728487.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_2["total_purchase"] = 0


,ID,item1,item2,item3,item4,item5,item6,item7,item8,item9,...,"item_2microsoft_office_home_and_student_2019,",item_kitchen_scales_&_measures,item_2targus_geolite_essential_case,item_2logitech_pebble_m350_bluetooth_mouse,item_bathroom,item_haircare,item_home_safety_equipment,item_childrens_footwear,item_men_s_sportswear,total_purchase
0,85517,COMPUTERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,889.0
1,51113,COMPUTER PERIPHERALS ACCESSORIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,409.0
2,83008,TELEVISIONS HOME CINEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1399.0
3,78712,COMPUTERS,COMPUTER PERIPHERALS ACCESSORIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,808.0
4,77846,TELEVISIONS HOME CINEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92785,21243,COMPUTER PERIPHERALS ACCESSORIES,FULFILMENT CHARGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,306.0
92786,45891,COMPUTERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,898.0
92787,42613,BEDROOM FURNITURE,BEDROOM FURNITURE,OUTDOOR FURNITURE,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1727.0
92788,43567,COMPUTERS,COMPUTERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,3198.0


In [8]:
# Here, as an example, you can verify that the mapping between item columns 1 to 24 and the
# column associated with computer_peripherals_accessories was successful
train_2[["item1","item_computer_peripherals_accessories","goods_code1"]]

,item1,item_computer_peripherals_accessories,goods_code1
0,COMPUTERS,0,239246776
1,COMPUTER PERIPHERALS ACCESSORIES,1,239001518
2,TELEVISIONS HOME CINEMA,0,239842093
3,COMPUTERS,1,239001422
4,TELEVISIONS HOME CINEMA,0,239952954
...,...,...,...
92785,COMPUTER PERIPHERALS ACCESSORIES,1,238905679
92786,COMPUTERS,0,239246776
92787,BEDROOM FURNITURE,0,236938427
92788,COMPUTERS,0,240040978


In [9]:
# We drop the itemx, Nbr_of_prod_purchasx, and cash_pricex variables
for y in range (1,25):
      train_2 = train_2.drop("item{0}".format(y),axis = 1)
      train_2 = train_2.drop("Nbr_of_prod_purchas{0}".format(y), axis = 1)
      train_2 = train_2.drop("cash_price{0}".format(y), axis = 1)

train_2

,ID,make1,make2,make3,make4,make5,make6,make7,make8,make9,...,"item_2microsoft_office_home_and_student_2019,",item_kitchen_scales_&_measures,item_2targus_geolite_essential_case,item_2logitech_pebble_m350_bluetooth_mouse,item_bathroom,item_haircare,item_home_safety_equipment,item_childrens_footwear,item_men_s_sportswear,total_purchase
0,85517,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,889.0
1,51113,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,409.0
2,83008,SAMSUNG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1399.0
3,78712,APPLE,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,808.0
4,77846,SONY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92785,21243,SAMSUNG,RETAILER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,306.0
92786,45891,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,898.0
92787,42613,SILENTNIGHT,SILENTNIGHT,LG OUTDOOR,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1727.0
92788,43567,APPLE,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,3198.0


### 2.- After analyzing the data, it was found that there are very similar products, for example:

* #### computer_peripherals_accessories
* #### computer_peripherals_&_accessories

### The proposal, then, is to use cosine similarity to compare fields with each other and, if they are similar, merge them (including their counts) into a single one, since they are essentially the same product.

### For the acceptance criterion, it was decided that with a 60% similarity threshold it can be considered the same field. This value was chosen by trial and error.


In [10]:
# To roughly see columns with possible similar names.
columns = train_2.columns

for column in columns:
    if "computer" in column:
        print(column)

item_computers
item_computer_peripherals_accessories
item_computer_peripherals_&_accessories
item_computer_software
item_computer_networking


In [11]:
#To compute similarity between two products, the following code was used as a basis
#the following code:
#https://stackoverflow.com/questions/8897593/how-to-compute-the-similarity-between-two-text-documents

def stem_tokens(tokens):
    """
    Stems a sequence of tokens.

    Args:
        tokens (list[str]): Token strings to be stemmed.

    Returns:
        list[str]: Stemmed token strings, in the same order.
    """
    return [stemmer.stem(item) for item in tokens]


def normalize(text):
    """
    Normalizes text by removing punctuation, lowercasing, tokenizing, and stemming.

    Steps:
        1) Lowercase the input string.
        2) Remove punctuation using `remove_punctuation_map`.
        3) Tokenize using `nltk.word_tokenize`.
        4) Stem tokens using `stem_tokens` (and the global `stemmer`).

    Args:
        text (str): Raw input text.

    Returns:
        list[str]: Normalized tokens (stemmed).
    """
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

def cosine_sim(text1, text2):
    """
    Computes cosine similarity between two texts using TF-IDF vectors.

    This fits the global `vectorizer` on `[text1, text2]`, transforms them into TF-IDF vectors,
    then returns the cosine similarity between the two vectors.

    Args:
        text1 (str): First input text.
        text2 (str): Second input text.

    Returns:
        float: Cosine similarity score in the range [0.0, 1.0] for non-negative TF-IDF vectors.

    Notes:
        - This refits `vectorizer` on every call, so the score depends only on these two texts.
        - Requires `vectorizer` to be a fitted-capable TF-IDF vectorizer (e.g., sklearn TfidfVectorizer).
    """
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).toarray() )[0, 1]

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

In [12]:
# In this operation, we iterate over each column name and compare it with the rest
# If there is a cosine similarity greater than 60%, then we conclude that the fields
# have a very similar name and they are merged.

# Getting the list of products.
current_items = train_2.columns

# To take into account columns already removed.
black_list = []

# To know how many variables were merged.
counter = 0

for x in range(1,len(current_items)):
    current_product = current_items[x]

    for y in range(x,len(current_items)):
        challenge_product = current_items[y]

        # Only compare columns with the "item" prefix
        # which are the ones created previously using the product-column approach.
        if "item" in current_product and "item" in challenge_product and x != y:

            # We emove the word "item" and underscores so we can compute similarity
            # over a sentence.
            current_product_vector = current_items[x].replace("item_","").replace("_"," ")
            challenge_product_vector = current_items[y].replace("item_","").replace("_"," ")

            # We compute similarity between two columns.
            similarity = cosine_sim(current_product_vector,challenge_product_vector)

            # If similarity exceeds the threshold, merge both columns and
            # drop the second one.
            if similarity > 0.60 and current_product not in black_list:
               train_2[current_product] = train_2[current_product] + train_2[challenge_product]
               train_2 = train_2.drop(challenge_product, axis = 1)
               black_list.append(challenge_product)
               print("      Current: {0} | Challenge: {1} | Similarity: {2}".format(current_product_vector,challenge_product_vector,similarity))
               print(" ")
               counter += 1

print("{0} variables were fusioned.".format(counter))
print(" ")
train_2

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'there

      Current: televisions home cinema | Challenge: televisions & home cinema | Similarity: 1.0000000000000002
 
      Current: computer peripherals accessories | Challenge: computer peripherals & accessories | Similarity: 1.0000000000000002
 


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


      Current: living dining furniture | Challenge: living & dining furniture | Similarity: 1.0000000000000002
 
      Current: telephones, fax machines & two-way radios | Challenge: telephones fax machines two-way radios | Similarity: 0.9999999999999999
 


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


...

      Current: microsoft office home and student 2019, | Challenge: 2microsoft office home and student 2019, | Similarity: 0.6694188517266485
 
      Current: targus geolite essential case | Challenge: 2targus geolite essential case | Similarity: 0.6029748160380572
 


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


      Current: logitech pebble m350 bluetooth mouse | Challenge: 2logitech pebble m350 bluetooth mouse | Similarity: 0.6694188517266485
 
40 variables were fusioned.
 


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,ID,make1,make2,make3,make4,make5,make6,make7,make8,make9,...,item_targus_geolite_essential_case,item_easter_decorations,item_logitech_pebble_m350_bluetooth_mouse,item_2hp_elitebook_850v6,item_paper_notebooks,item_bathroom,item_haircare,item_home_safety_equipment,item_men_s_sportswear,total_purchase
0,85517,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,889.0
1,51113,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,409.0
2,83008,SAMSUNG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1399.0
3,78712,APPLE,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,808.0
4,77846,SONY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92785,21243,SAMSUNG,RETAILER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,306.0
92786,45891,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,898.0
92787,42613,SILENTNIGHT,SILENTNIGHT,LG OUTDOOR,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1727.0
92788,43567,APPLE,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,3198.0


### 3.- For this step, the X & y sets are joined to, among other things, see whether the variables make1 through make24 are useful, as well as model1 through model24, and also goods_code1 through goods_code24.

### The reasoning is as follows:

### Take make1 and check the most frequent values for both those who commit fraud and those who do not. If the values have similar frequencies, then it makes no sense to consider this variable because it is not discriminating fraud.

### The same applies to model1 and goods_code1. In general, these variables are used because they contain 90% of the data; the rest mostly contains nulls.

### De manera secundaria, se eliminan tanto la variable index que viene del conjunto Y, la variable ID puesto que ya no es necesaria dado que se ha unido ya el conjunto de datos y desde el punto de vista Data-Centric, para el modelo no aporta nada, y la variable NB_items pues ya se tiene como resultado de los productos adquiridos.

In [13]:
# To identify how the distribution of fraudulent and non-fraudulent cases looks.
target["fraud_flag"].value_counts()

fraud_flag
0    91471
1     1319
Name: count, dtype: int64

In [14]:
# We merge X and y sources for training.
final_source = target.merge(train_2,on="ID")

In [15]:
# Next is the analysis for model1 for those who do NOT commit fraud.
print(final_source[final_source["fraud_flag"] == 0 ] ["model1"].value_counts().to_string())

model1
2020 APPLE MACBOOK AIR 13 3 RETINA DISPLAY M1 PROC    12563
2020 APPLE MACBOOK PRO 13 TOUCH BAR M1 PROCESSOR 8     5008
2021 APPLE MACBOOK PRO 14 M1 PRO PROCESSOR 16GB RA     3365
RETAILER                                               2582
2020 APPLE IPAD AIR 10 9 A14 BIONIC PROCESSOR IOS      2421
2021 APPLE IMAC 24 ALL-IN-ONE M1 PROCESSOR 8GB RAM     2329
2021 APPLE MACBOOK PRO 16 M1 PRO PROCESSOR 16GB RA     2037
2021 APPLE IPAD PRO 11 M1 PROCESSOR IOS WI-FI 128G     2028
2020 APPLE MACBOOK AIR                                 1963
2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-FI 25     1796
APPLE WATCH SERIES 6                                   1713
LG OLED55C14LB 2021 OLED HDR 4K ULTRA HD SMART TV      1681
2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-FI 12     1652
APPLE WATCH SERIES 7 GPS 45MM MIDNIGHT ALUMINIUM C     1440
APPLE WATCH SERIES 7 GPS 41MM STARLIGHT ALUMINIUM      1328
LG OLED48C14LB 2021 OLED HDR 4K ULTRA HD SMART TV      1188
APPLE WATCH SERIES 7 GPS 41MM MID

In [16]:
# Next is the analysis for model1 for those who DO commit fraud.
print(final_source[final_source["fraud_flag"] == 1 ] ["model1"].value_counts().to_string())

model1
2020 APPLE MACBOOK PRO 13 TOUCH BAR M1 PROCESSOR 8    164
2021 APPLE MACBOOK PRO 14 M1 PRO PROCESSOR 16GB RA    150
2020 APPLE MACBOOK AIR 13 3 RETINA DISPLAY M1 PROC    142
2021 APPLE MACBOOK PRO 16 M1 PRO PROCESSOR 16GB RA     74
2021 APPLE IPAD PRO 11 M1 PROCESSOR IOS WI-FI 128G     59
2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-FI CE     55
2020 APPLE MACBOOK PRO 13 TOUCH BAR INTEL CORE I5      54
APPLE IPHONE 12 PRO MAX                                36
2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-FI 25     35
2021 APPLE IPAD PRO 11 M1 PROCESSOR IOS WI-FI CELL     32
2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-FI 12     32
2019 APPLE MACBOOK PRO 16 TOUCH BAR INTEL CORE I9      26
2020 APPLE MACBOOK PRO                                 25
2021 APPLE IPAD PRO 12 9 M1 PROCESSOR IOS WI-FI 51     21
APPLE IPHONE 12 PRO                                    21
2021 APPLE IMAC 24 ALL-IN-ONE M1 PROCESSOR 8GB RAM     20
2022 APPLE IPAD AIR 10 9 M1 PROCESSOR IPADOS WI-FI     18
APPLE I

In [17]:
# Next is the analysis for make1 for those who do NOT commit fraud.
print(final_source[final_source["fraud_flag"] == 0 ] ["make1"].value_counts().to_string())

make1
APPLE                             62682
LG                                 8095
RETAILER                           6032
SAMSUNG                            4636
SONY                               2966
PANASONIC                           383
KETTLER                             369
ANYDAY RETAILER                     366
WEST ELM                            341
DYSON                               316
SWOON                               290
HALO                                262
SWYFT                               214
INNOVATION LIVING                   211
TEMPUR                              169
BUGABOO                             155
HYPNOS                              154
HERMAN MILLER                       153
G PLAN VINTAGE                      116
WEBER                               111
MICROSOFT                           105
SILVER CROSS                         96
LG OUTDOOR                           84
BIG GREEN EGG                        80
EGG                               

In [18]:
# Next is the analysis for make1 for those who DO commit fraud.
print(final_source[final_source["fraud_flag"] == 1] ["make1"].value_counts().to_string())

make1
APPLE              1186
SAMSUNG              62
LG                   18
SONY                  7
RETAILER              5
SILVER CROSS          5
PANASONIC             5
BUGABOO               3
KETTLER               2
PHILIPS               2
LEVI S                2
TOMMEE TIPPEE         1
RALPH LAUREN          1
MAXI-COSI             1
CYBEX                 1
NIKON                 1
GAIA BABY             1
BT                    1
HALO                  1
TEMPUR                1
SHNUGGLE              1
MORPHY RICHARDS       1
RING                  1
LE CREUSET            1
KARTELL               1
DELL                  1


In [19]:
# Next is the analysis for make1 for those who do NOT commit fraud.
print(final_source[final_source["fraud_flag"] == 0] ["goods_code1"].value_counts().to_string())

goods_code1
239246776    4508
239246779    2338
239246778    2144
239246776    1944
239246782    1693
240575990    1535
239827061    1155
240040984    1152
239246775    1115
239246779    1090
240040978    1067
240041004    1003
240376619     994
239246783     961
240376608     921
239827062     887
239246778     860
240376609     776
239838687     764
239246782     756
240575993     716
240575990     637
239001518     573
239827061     568
239001474     535
240040984     535
239246780     513
239246775     506
240040978     506
240041020     479
237841896     470
240041004     470
239246783     447
240376619     446
239001428     444
239900855     414
240376608     407
240575999     404
239001415     403
239246777     402
239827062     394
240575992     388
239866722     370
239001536     368
238601186     363
239246774     359
238742046     358
240575996     357
240376609     326
240041012     316
240040968     314
240575993     304
239246781     302
239838687     293
239841491     29

In [20]:
# Next is the analysis for make1 for those who DO commit fraud.
print(final_source[final_source["fraud_flag"] == 1] ["goods_code1"].value_counts().to_string())

goods_code1
239246779    75
240575990    69
239246776    65
239246783    46
239246783    31
240575990    27
240040984    27
239246779    25
240040978    25
239246776    20
240575993    19
238742053    19
240575992    19
240575993    19
238449039    18
240041004    17
239246778    16
239246775    15
238742046    15
240575996    15
240040984    14
240041012    14
240041010    14
240041004    13
238449039    13
239246782    12
239091971    11
240575999    10
240041013    10
240575999    10
240040997    10
238742052     9
239091980     9
238742046     9
240040997     9
239246778     8
240040999     8
240041016     8
240575997     8
238742053     7
240575996     7
240575992     7
240040999     7
239091963     7
239246773     6
240040988     6
240040978     6
240041020     5
239092000     5
239246780     5
240040992     5
240040987     4
240041022     4
240040994     4
239246781     4
238449041     4
240040963     4
238449040     4
239246774     4
240041016     4
240040983     4
240243877   

In [21]:
# Here we only show that goods_code mostly belong to the same type (Apple or computers)
# so they are not very representative.
train_2[train_2["goods_code1"].isin([239246779,240575990,239246776,239246783,239246783,240040984,240575990])]

,ID,make1,make2,make3,make4,make5,make6,make7,make8,make9,...,item_targus_geolite_essential_case,item_easter_decorations,item_logitech_pebble_m350_bluetooth_mouse,item_2hp_elitebook_850v6,item_paper_notebooks,item_bathroom,item_haircare,item_home_safety_equipment,item_men_s_sportswear,total_purchase
0,85517,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,889.0
9,39361,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,898.0
14,70262,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1899.0
15,108706,APPLE,APPLE,RETAILER,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,843.0
18,70897,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,749.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92758,7877,APPLE,RETAILER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,2304.0
92766,82457,APPLE,RETAILER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1194.0
92769,108631,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,887.0
92783,97639,APPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1855.0


### As you can see, in general for both make and model the most frequent values tend to be the same for people who commit fraud and those who do not. Even more, for goods_code the most frequent values are related to computers, so it is not very representative either.

### Therefore, we can remove such variables.

In [22]:
# Drop the modelx and makex variables.
for z in range (1,25):
      final_source = final_source.drop("model{0}".format(z),axis = 1)
      final_source = final_source.drop("make{0}".format(z), axis = 1)
      final_source = final_source.drop("goods_code{0}".format(z), axis = 1)

final_source = final_source.drop("index", axis = 1)
final_source = final_source.drop("ID", axis = 1)
final_source = final_source.drop("Nb_of_items", axis = 1)

final_source

,fraud_flag,item_computers,item_televisions_home_cinema,item_computer_peripherals_accessories,item_living_dining_furniture,"item_telephones,_fax_machines_&_two-way_radios",item_bedroom_furniture,item_outdoor_furniture,item_baby_child_travel,item_audio_accessories,...,item_targus_geolite_essential_case,item_easter_decorations,item_logitech_pebble_m350_bluetooth_mouse,item_2hp_elitebook_850v6,item_paper_notebooks,item_bathroom,item_haircare,item_home_safety_equipment,item_men_s_sportswear,total_purchase
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,889.0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,409.0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1399.0
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,808.0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92785,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,306.0
92786,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,898.0
92787,0,0,0,0,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,1727.0
92788,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3198.0


### 4.- For the next operation, correlation analyses are performed in two stages:

### a) Between the predictor variables. In this case we want correlation as close to 0 as possible.

### b) Between the predictor variables and the predictive variable. In this case we want correlation as close to 1 as possible. The threshold used at this point is 0.01, chosen by trial and error.

<br>

### Once the variables that meet the mentioned conditions are identified, they are removed from the dataset.

In [23]:
def correlation(dataframe, graphic=True, matrix=False):
    """
    Computes the correlation matrix for a dataframe and optionally plot it as a heatmap.

    Args:
        dataframe (pandas.DataFrame): Input data used to compute pairwise correlations
            via `dataframe.corr()` (typically numeric columns only).
        graphic (bool): If True, display a seaborn heatmap of the correlation matrix.
        matrix (bool): If True, return the correlation matrix.

    Returns:
        pandas.DataFrame or None: The correlation matrix if `matrix` is True, otherwise None.

    Notes:
        - This function uses `plt` (matplotlib) and `sns` (seaborn) from the surrounding scope.
        - If `graphic` is True, the heatmap is created but not explicitly shown with `plt.show()`.
        - If both `graphic` and `matrix` are False, the function computes the matrix but returns nothing.
    """
    corr = dataframe.corr()
    if graphic == True:
       fig = plt.figure(figsize=(25, 20))
       ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0, cmap="coolwarm", annot=True, fmt=".2f", square=True)
       ax.set_xticklabels(ax.get_xticklabels(), rotation=45,horizontalalignment='right')
        
    if matrix == True:
        return corr


In [24]:
# We compute the correlation matrix between predictor variables; the analysis here is direct since the function
# provided in class helps with that.

# Only one detail is added: since there are many variables, printing it visually exhausts computational resources,
# so we only compute the value without plotting.
corr_matrix = correlation(final_source[[c for c in final_source if c != predictive_variable ]], graphic=False, matrix=True)

In [25]:
# We get the upper triangle of the correlation matrix in absolute values.
cell_above_main_diagonal = np.triu(np.ones_like(corr_matrix),1).astype(bool)
upper_triangle = corr_matrix.where(cell_above_main_diagonal).abs()

predictor_threshold = 1

# Find the columns where the absolute correlation is 1.
predictor_variables_to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] >= predictor_threshold)]

print("The number of variables with absolute correlation = {0} is: {1}".format(predictor_threshold,len(predictor_variables_to_drop)))
print("The variables: ")
for element in predictor_variables_to_drop:
    print(element)

The number of variables with absolute correlation = 1 is: 4
The variables: 
item_microsoft_office_home_and_student_2019,
item_targus_geolite_essential_case
item_logitech_pebble_m350_bluetooth_mouse
item_2hp_elitebook_850v6


In [26]:
# We generate the correlation analysis between predictor variables and the predictive variable.
predictive_threshold = 0.01

df_corr_2 = final_source.corr()[predictive_variable].sort_values(ascending = False)

# Here we only prettify the result to see if there are variables with correlation 0 with the
#predictive variable.

# We drop the correlations closest to 0 since they add little to the analysis.
df_corr_2 = df_corr_2.to_frame()
df_corr_2.reset_index(inplace=True)
df_corr_2.columns = ['Variable','Correlation']

# We take the absolute value to build the filters.
df_corr_2["Correlation_Abs"] = np.abs(df_corr_2["Correlation"])

print("Correlations with the predictive variable: ")
print(df_corr_2.to_string())

# less than or equal to 0.001 abs
df_filter = df_corr_2[df_corr_2["Correlation_Abs"] <= predictive_threshold]

print(" ")
print(" ")

print("Variables to remove due to low correlation with the predictive variable: ")
print(df_filter.to_string())

# Get the variables to drop relative to the predictive variable.
predictive_variables_to_drop = df_filter["Variable"].tolist()

Correlations with the predictive variable: 
                                           Variable  Correlation  Correlation_Abs
0                                        fraud_flag     1.000000         1.000000
1                                    item_computers     0.078039         0.078039
2                                    total_purchase     0.048674         0.048674
3                            item_fulfilment_charge     0.040885         0.040885
4                            item_audio_accessories     0.039096         0.039096
5    item_telephones,_fax_machines_&_two-way_radios     0.025072         0.025072
6                            item_imaging_equipment     0.016662         0.016662
7                                      item_luggage     0.011369         0.011369
8                      item_kitchen_scales_measures     0.011169         0.011169
9                     item_kitchen_utensils_gadgets     0.010739         0.010739
10                               item_men_s_clothes   

In [27]:
# Dropping predictor variables.
for element in predictor_variables_to_drop:
    try:
       final_source = final_source.drop(element,axis = 1)
    except:
       print("Variable {0} already deleted".format(element))

# Dropping variables relative to the predictive variable.
for element2 in predictive_variables_to_drop:
    try:
        final_source = final_source.drop(element2,axis = 1)
    except:
       print("Variable {0} already deleted".format(element2))

final_source

Variable item_targus_geolite_essential_case already deleted
Variable item_2hp_elitebook_850v6 already deleted
Variable item_logitech_pebble_m350_bluetooth_mouse already deleted
Variable item_microsoft_office_home_and_student_2019, already deleted


,fraud_flag,item_computers,item_televisions_home_cinema,item_computer_peripherals_accessories,item_living_dining_furniture,"item_telephones,_fax_machines_&_two-way_radios",item_bedroom_furniture,item_outdoor_furniture,item_audio_accessories,item_cables_adapters,item_bags_carry_cases,item_imaging_equipment,item_kitchen_utensils_gadgets,item_luggage,item_fulfilment_charge,item_warranty,item_service,item_kitchen_scales_measures,total_purchase
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,889.0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,409.0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1399.0
3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,808.0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1199.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92785,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,306.0
92786,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,898.0
92787,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,1727.0
92788,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3198.0


In [28]:
# We compute some statistics for the end.
final_source.describe()

,fraud_flag,item_computers,item_televisions_home_cinema,item_computer_peripherals_accessories,item_living_dining_furniture,"item_telephones,_fax_machines_&_two-way_radios",item_bedroom_furniture,item_outdoor_furniture,item_audio_accessories,item_cables_adapters,item_bags_carry_cases,item_imaging_equipment,item_kitchen_utensils_gadgets,item_luggage,item_fulfilment_charge,item_warranty,item_service,item_kitchen_scales_measures,total_purchase
count,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000,92790.000000
mean,0.014215,0.541233,0.160664,0.217556,0.100593,0.050124,0.051708,0.018903,0.017998,0.016866,0.012394,0.000916,0.001724,0.000162,0.269673,0.088512,0.036372,0.000151,1234.759554
std,0.118376,0.511112,0.380426,0.452858,0.451402,0.224723,0.323155,0.189863,0.135671,0.139071,0.111894,0.031645,0.073972,0.014309,0.443793,0.290900,0.211540,0.014681,771.484197
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,219.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,779.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1099.000000
75%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1499.000000
max,1.000000,4.000000,3.000000,5.000000,12.000000,3.000000,8.000000,7.000000,3.000000,4.000000,3.000000,2.000000,9.000000,2.000000,1.000000,3.000000,4.000000,2.000000,21995.000000


In [29]:
# Save the final dataset source.
final_source.to_csv("results/DataPubDataCentricChallenge_AaronMartinCastilloMedina.csv",index=False)